In [25]:
# Importar as bibliotecas necessárias

import os
from dotenv import load_dotenv
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import panel as pn

In [ ]:
# Carregar as variáveis de ambiente do arquivo .env

load_dotenv()

In [27]:
# Pegar os valores das variáveis de ambiente

db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')

DATABASE_URL = os.getenv('DATABASE_URL')

In [ ]:
# Conectar ao banco de dados PostgreSQL

try:
    # Estabelecendo conexão com o driver

    conn = psycopg2.connect(
        dbname=db_name,
        user=db_user,
        password=db_password,
        host=db_host,
        port=db_port
    )

    # Estabelecendo conexão com o SQLAlchemy

    engine = create_engine(DATABASE_URL)

    print("Conexão estabelecida com sucesso!")
    
except Exception as e:
    print(f"Erro ao conectar ao banco de dados: {e}")

In [ ]:
pn.extension()
pn.extension('tabulator')
pn.extension(notifications=True)

In [ ]:
flag = ""

game_name = pn.widgets.TextInput(
    name="Nome do Jogo",
    value="",
    placeholder="Digite o nome do jogo",
    disabled=False
)

buttonSelect = pn.widgets.Button(name="Consultar", button_type="default")
buttonInsert = pn.widgets.Button(name="Inserir", button_type="default")
buttonDelete = pn.widgets.Button(name="Excluir", button_type="default")

In [ ]:
# Funções para manipular o banco de dados
def select_game(event):
    game_name = game_name.value
    query = f"SELECT * FROM games WHERE ('{game_name.value_input}'='{flag} or name = '{game_name.value_input}')"
    try:
        df = pd.read_sql(query, con=engine)
        pn.state.notifications.success("Consulta realizada com sucesso!")
        return df
    except Exception as e:
        pn.state.notifications.error(f"Erro ao consultar: {e}")

def insert_game(event):
    game_name = game_name.value
    try:
        query = f"INSERT INTO games (name) VALUES ('{game_name}')"
        with engine.connect() as connection:
            connection.execute(query)
        pn.state.notifications.success(f"Jogo '{game_name}' inserido com sucesso!")
    except Exception as e:
        pn.state.notifications.error(f"Erro ao inserir: {e}")

def delete_game(event):
    game_name = game_name.value
    try:
        query = f"DELETE FROM games WHERE name = '{game_name}'"
        with engine.connect() as connection:
            connection.execute(query)
        pn.state.notifications.success(f"Jogo '{game_name}' excluído com sucesso!")
    except Exception as e:
        pn.state.notifications.error(f"Erro ao excluir: {e}")

# Associar funções aos botões
buttonSelect.on_click(select_game)
buttonInsert.on_click(insert_game)
buttonDelete.on_click(delete_game)

In [ ]:
layout = pn.Column(
    game_name,
    pn.Row(buttonSelect, buttonInsert, buttonDelete)
)

pn.serve(layout)